In [1]:
setwd("/liulab/galib/dlbcl_manuscript/")
library(tidyverse)
library(Seurat)
library(harmony)
library(dplyr)
library(DoubletFinder)
library(rBCS)

Warning message:
“package ‘tidyverse’ was built under R version 4.1.3”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.0      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.1 

Warning message:
“package ‘tidyr’ was built under R version 4.1.2”
Warning message:
“package ‘readr’ was built under R version 4.1.2”
Warning message:
“package ‘forcats’ was built under R version 4.1.3”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching SeuratObject

Attaching sp

Warning message:
“package ‘harmony’ was built under R version 4.1.3”
Loading required package: Rcpp

Warning message:
“package ‘Rcpp’ was built under R version 4.1.2”
Warning message:
“package ‘DoubletFinder’ was built under R version 4.1.3”
Warning message:
“package ‘rBCS’ was built under R v

### Read count matrices and build seurat object

In [ ]:
scRNAseq_meta<- read_xlsx("./data/scRNAseq.xlsx")

scRNAseq_meta<- scRNAseq_meta %>%
  clean_names() %>%
  rename(sample_id = number)

output_folder<- "./data/GEXdata/"

cellranger_dirs<- list.dirs(output_folder, recursive = TRUE, full.names = TRUE)
cellranger_dirs<- str_subset(cellranger_dirs, "filtered_feature_bc_matrix$")

samples<- map_chr(cellranger_dirs, ~ str_split( .x, pattern= "/", simplify = TRUE)[8])

scRNAseq_meta<- scRNAseq_meta %>% left_join(tibble(dir = cellranger_dirs, pool_id = samples))

create_from_dir<- function(meta, min.cells = 0, min.features = 0){

  dir<- pull(meta, dir)
  sample<- pull(meta, pool_id)
  mat<- Read10X(data.dir = dir)
  colnames(mat)<- paste(sample,colnames(mat), sep="-")
  obj<- CreateSeuratObject(counts = mat, min.cells = min.cells, min.features = min.features)
  cell_number<- ncol(obj)

  old_meta<- obj@meta.data

  #create a dataframe with the same number of rows as the cell number
  additional_meta<- replicate(cell_number, meta, simplify = FALSE) %>% bind_rows()
  new_meta<- cbind(old_meta, additional_meta)

  obj@meta.data<- new_meta
  obj$orig.ident<- obj$pool_id
  return(obj)
}

seurat_objects<- map(1:nrow(scRNAseq_meta), ~ create_from_dir(scRNAseq_meta[.x, ]))
merged_seurat<- merge(seurat_objects[[1]], seurat_objects[-1])

merged_seurat[["percent.mt"]] <- PercentageFeatureSet(merged_seurat, pattern = "^mt-")
merged_seurat$orig.ident<- paste0("mouse", merged_seurat$sample_id)
Idents(merged_seurat)<- merged_seurat$orig.ident

VlnPlot(merged_seurat, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 1,
        pt.size = 0)

ggsave("./results/figures/0_original_mito_percentage.pdf", width = 24, height = 8)

### Run harmony for correcting batch effects

In [ ]:
library(harmony)
merged_seurat<- merged_seurat %>%
  NormalizeData(normalization.method = "LogNormalize", scale.factor = 10000) %>%
  FindVariableFeatures(selection.method = "vst", nfeatures = 2000) %>%
  ScaleData(vars.to.regress = "percent.mt") %>%
  RunPCA(npc = 50)


merged_seurat <-  RunHarmony(merged_seurat, group.by.vars = "pool_id") %>%
  RunUMAP(reduction = "harmony", dims = 1:50) %>%
  FindNeighbors(reduction = "harmony", dims = 1:50) %>%
  FindClusters(resolution = 1.5)


#32285 genes x 411851 cells
saveRDS(merged_seurat, "./data/objects/merged_mouse_seurat_harmony_res1.5.rds")

### Remove bad quality cells

In [8]:
print("Reading merged object...")
# merged_seurat <- readRDS("./data/objects/merged_mouse_seurat_harmony_res1.5.rds")

print("Start to QC...200< nFeature_RNA <5000 & percent.mt < 10")

merged <- subset(merged_seurat, subset = nFeature_RNA > 200 & nFeature_RNA < 5000 & percent.mt < 10)
# 32285 genes x 390342 cells: 5% of the cells were filtered out

merged$project <- "Cd70_mouse"

VlnPlot(merged, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), group.by = "project", ncol = 3,
        pt.size = 0)
ggsave("./results/figures/0_merged_mito_percentage_10_nfeature_200_after_removal.pdf", width = 24, height = 8)

[1] "Reading merged object..."
[1] "Start to QC...200< nFeature_RNA <5000 & percent.mt < 10"


### Remove doublets with DoubletFinder

In [ ]:
preprocess_seurat<- function(obj){
  obj <- obj %>%
    NormalizeData(normalization.method = "LogNormalize", scale.factor = 10000) %>%
    FindVariableFeatures(selection.method = "vst", nfeatures = 2000) %>%
    ScaleData(vars.to.regress = "percent.mt") %>%
    RunPCA(npc = 50) %>%
    RunHarmony(group.by.vars = "pool_id") %>%
    RunUMAP(reduction = "harmony", dims = 1:50) %>%
    FindNeighbors(reduction = "harmony", dims = 1:50) %>%
    FindClusters(resolution = 1.5)
  return(obj)
}

print("processing merged object...")

merged<- preprocess_seurat(merged)


print("Start Doublet Finder...")

find_doublet<- function(obj){
  obj<- preprocess_seurat(obj)
  sweep.res <- paramSweep_v3(obj, PCs = 1:50, sct = FALSE)
  sweep.stats <- summarizeSweep(sweep.res, GT = FALSE)
  bcmvn <- find.pK(sweep.stats)

  ## Homotypic Doublet Proportion Estimate     -------------------------------------------------------------------------------------
  annotations<- obj$RNA_snn_res.1.5
  homotypic.prop <- modelHomotypic(annotations)

  number_of_cells<- nrow(obj@meta.data)
  nExp_poi<- round((number_of_cells/1000) * 0.008 * number_of_cells)
  nExp_poi.adj <- round(nExp_poi*(1-homotypic.prop))
  obj <- doubletFinder_v3(obj, PCs = 1:50, pN = 0.25, pK = 0.09, nExp = nExp_poi.adj, reuse.pANN = FALSE, sct = FALSE)
  return(obj)
}


print("Spliting data by poolID...")
merged_list <- SplitObject(merged, split.by = "pool_id")

print("Finding Doublets...")
merged_finddoublet_list <- lapply(merged_list, find_doublet)

print("Saving DoubletsFinder results...")
saveRDS(merged_finddoublet_list, "./data/objects/merged_DoubletFinder.rds")


#rename classification
rename_seurat_metadata<- function(obj){
    old.data<- obj@meta.data
    new.data<- old.data %>% 
        dplyr::rename_with(function(x) "DF.classifications", starts_with("DF.classifications"))
    obj@meta.data<- new.data
    return(obj)
}


merged_finddoublet_list2<- lapply(merged_finddoublet_list, rename_seurat_metadata)

doublet_meta<- bind_rows(lapply(merged_finddoublet_list2, function(x) x@meta.data))
merged@meta.data$DF.classifications <- doublet_meta$DF.classifications


#rename doublet score
rename_seurat_metadata_2<- function(obj){
    old.data<- obj@meta.data
    new.data<- old.data  %>%  
        dplyr::rename_with(function(x) "pANN_DoubletFinder", starts_with("pANN_"))
    obj@meta.data<- new.data
    return(obj)
}
                           
                           
merged_finddoublet_list3<- lapply(merged_finddoublet_list, rename_seurat_metadata_2)

doublet_meta<- bind_rows(lapply(merged_finddoublet_list3, function(x) x@meta.data))
merged@meta.data$pANN_DoubletFinder <- doublet_meta$pANN_DoubletFinder

print("Saving doubelts meta...")
write_tsv(merged@meta.data, "./results/tsv/merged/merged_DoubletFinder_doublets_meta.tsv")
# 332285 genes x 390342 cells
saveRDS(merged, "./data/objects/merged_qc_before_dbl.rds")

##### To start new session
# merged = readRDS("./data/objects/merged/merged_qc_before_dbl.rds")
# meta <- read_tsv("./results/tsv/merged/merged_DoubletFinder_doublets_meta.tsv")
# merged@meta.data$pANN_DoubletFinder <- meta$pANN_DoubletFinder
# merged@meta.data$DF.classifications <- meta$DF.classifications

table(merged@meta.data$`RNA_snn_res.1.5`, merged$DF.classifications) %>% unclass() %>%
  as.data.frame() %>%
  tibble::rownames_to_column(var="cluster") %>%
  mutate(percent = Doublet/(Doublet + Singlet)) %>%
  ggplot(aes(x= reorder(cluster, -Doublet), y = Doublet)) +
  geom_col() +
  theme_classic(base_size = 14) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) +
  xlab("") +
  ggtitle("Doublet counts in all cells clusters")

ggsave("results/figures/0_merged_DoubletFinder_doublet_number_percluster.pdf", width = 12, height = 8)


table(merged@meta.data$`RNA_snn_res.1.5`, merged$DF.classifications) %>% unclass() %>%
  as.data.frame() %>%
  tibble::rownames_to_column(var="cluster") %>%
  mutate(percent = Doublet/(Doublet + Singlet)) %>%
  ggplot(aes(x= reorder(cluster, -percent), y = percent)) +
  geom_col() +
  scale_y_continuous(labels = scales::percent) +
  theme_classic(base_size = 14) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) +
  xlab("") +
  ggtitle("Doublet percentage in all cells clusters")

ggsave("results/figures/0_merged_DoubletFinder_doublet_percent_percluster.pdf", width = 12, height = 8)


DimPlot(merged, group.by = "RNA_snn_res.1.5",
        label = TRUE, pt.size = 0.4 ) + labs(title = "UMAP by seurat clusters res =1.5", y = NULL,
                                             x = NULL) +  theme(text = element_text(size = 20)) + NoLegend()
ggsave("results/figures/0_merged_seurat_clusters_res1.5_umap.pdf", width = 12, height = 8)


DimPlot(merged, group.by = "DF.classifications",  cols = c('red4', 'gray'),
        label = FALSE, pt.size = 0.4 ) + labs(title = "UMAP by doublets", y = NULL,
                                              x = NULL) + theme(text = element_text(size = 20))
ggsave("results/figures/0_merged_DoubletFinder_Doublets_class_res1.5_umap.pdf", width = 12, height = 8)


FeaturePlot(merged, features = "pANN_DoubletFinder") + labs(title = "Doublets Score", y = NULL,
                                                                x = NULL) + theme(text = element_text(size = 20))

ggsave("results/figures/0_merged_DoubletFinder_Doublets_Score_res1.5_umap.pdf", width = 12, height = 8)


print("Removing Doublets...")
merged$DF.classifications  %>% table()

merged_final<- subset(merged, subset = DF.classifications != "Doublet")

saveRDS(merged_final, "data/objects/merged/merged_qc_doublet_rm.obj")
# 32285 genes x 376307 cells (3.6 % of cells removed)
dim(merged_final)
print("Doublets removal done!")